In [1]:
from init_nb import init_notebook
config = init_notebook()

import pandas as pd
from pathlib import Path
from bold_dementia.data.memento import Memento
from bold_dementia.connectivity.atlases import Atlas

/homes_unix/jlegrand/miniconda3/envs/memento/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dd = Path(config["data_dir"])

In [14]:
raw_kwargs = dict(encoding="unicode_escape", sep="\t", index_col=None)
base = Memento.load_phenotypes(
    dd / "data_memento.txt",
    date_format="%d/%m/%Y",
    timepoints=("IRM_M0", "IRM_M24", "IRM_M48"),
    **raw_kwargs
)
suite = Memento.load_phenotypes(
    dd / "data_memento_suite.txt",
    date_format="%d/%m/%Y",
    timepoints=("IRM_M0", "IRM_M24", "IRM_M48"),
    **raw_kwargs
)
merged= pd.concat([base, suite])
merged.to_csv(dd / "merged_phenotypes.csv")

In [15]:
pd.read_csv(dd /"merged_phenotypes.csv",index_col=0)

,CEN_ANOM,NUM_ID,SEX,INCCONSDAT_D,AGE_CONS,NIVETUD,MCI,CDRSCR,MA,DEMENCE_DAT,APOE_eps4,APOE_geno,APOE_y,M000,M024,M048,sub
0,EFW,SUBJ0001,Masculin,2013-10-14,79.6,<= BEPC,MCI non amnésique pur,0,NaN,NaN,0.0,eps3/eps3,Non,2013-12-06,2015-10-14,2017-10-16,1
1,JGX,SUBJ0003,Féminin,2013-04-30,74.4,>BAC,MCI amnésique multidomaine,0.5,NaN,NaN,0.0,eps3/eps3,Non,2013-07-09,2015-06-02,NaN,3
2,QVM,SUBJ0004,Féminin,2013-11-13,70.2,<= BEPC,MCI amnésique multidomaine,0.5,NaN,NaN,1.0,eps3/eps4,Oui,2014-02-04,2016-01-19,NaN,4
3,JGX,SUBJ0005,Féminin,2013-10-15,77.6,>BAC,MCI non amnésique multidomaine,0.5,NaN,NaN,0.0,eps3/eps3,Non,2013-12-12,2015-11-04,2017-10-11,5
4,NRR,SUBJ0006,Féminin,2012-06-25,74.1,>BAC,MCI non amnésique pur,0.5,NaN,NaN,1.0,eps3/eps4,Oui,2012-07-17,NaN,NaN,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,YLF,SUBJ2435,Féminin,2013-09-06,84.4,<= BEPC,MCI amnésique multidomaine,0.5,NaN,NaN,1.0,eps3/eps4,Oui,2013-10-17,2015-11-12,NaN,2435
365,EFW,SUBJ2438,Masculin,2013-05-31,72.0,"BEP, CAP, BAC",MCI non amnésique pur,0.5,NaN,NaN,0.0,eps3/eps3,Non,2013-07-12,2015-05-28,2017-07-26,2438
366,NRR,SUBJ2443,Féminin,2013-11-29,57.3,"BEP, CAP, BAC",MCI amnésique multidomaine,0.5,NaN,NaN,1.0,eps3/eps4,Oui,2014-01-21,2016-01-12,2017-12-12,2443
367,ISR,SUBJ2448,Féminin,2013-11-18,75.8,"BEP, CAP, BAC",MCI non amnésique pur,0,NaN,NaN,0.0,eps3/eps3,Non,2014-03-04,NaN,NaN,2448


In [42]:
BIDSDIR = Path(config["bids_dir"])
ppath = dd / "merged_phenotypes.csv"

memento = Memento(
        BIDSDIR,
        ppath,
        atlas=Atlas.from_name("gillig", False),
    )
df = Memento.load_phenotypes(ppath)

In [43]:
import numpy as np

test, retest = np.array_split(
    df.groupby("sub").sample(frac=1, random_state=config["seed"]), 2
)

/homes_unix/jlegrand/miniconda3/envs/memento/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [24]:
test.to_csv(dd / "test.csv")
retest.to_csv(dd / "retest.csv")